In [ ]:
# 安装依赖（仅需执行一次）
import subprocess
import sys

def install_dependencies():
    dependencies = [
        "torch", "torchvision", "numpy", "pandas", "matplotlib",
        "scikit-learn", "tqdm", "pickle-mixin"
    ]
    for dep in dependencies:
        subprocess.check_call([sys.executable, "-m", "pip", "install", dep])

# 执行安装（首次运行时取消注释）
# install_dependencies()
print("依赖安装完成（若未安装，取消上方注释执行）")

In [33]:
import os
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from tqdm import tqdm

# 中文显示配置
plt.rcParams['font.sans-serif'] = ['SimHei']  # 支持中文
plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示问题
plt.rcParams['figure.figsize'] = (15, 10)     # 默认图表大小
plt.rcParams['font.size'] = 10                # 默认字体大小

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备：{device}")
if device.type == 'cuda':
    print(f"GPU型号：{torch.cuda.get_device_name(0)}")
    print(f"GPU内存：{torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# ========== 关键配置 ==========
PKL_2D_DIR = '/root/second/2D'  # 你的2D pkl文件夹
PKL_3D_DIR = '/root/second/3D'  # 你的3D pkl文件夹
MODALITY = '3d'  # 选择检测模态：'2d' 或 '3d'（3D更精准）
MAX_FRAMES = 32  # 统一帧长（不足补0，超出截断）
SUPPORTED_FRAMES = [32, 64]  # 支持的原始帧数
BATCH_SIZE = 4
EPOCHS = 30
ANOMALY_THRESHOLD = 0.8  # 异常帧阈值（0-1，越大越严格，基于概率）
NUM_JOINTS = 71  # 骨架关节数（根据你的数据调整，原代码默认71）

# 输出目录（保存异常帧结果和可视化）
OUTPUT_DIR = 'asd_anomaly_detection_results_frame_level'
os.makedirs(OUTPUT_DIR, exist_ok=True)

ModuleNotFoundError: No module named 'cfg'

In [20]:
# 补充缺失的get_pkl_files函数
def get_pkl_files(dir_path):
    """获取目录下所有pkl文件"""
    return [os.path.join(dir_path, f) for f in os.listdir(dir_path) if f.endswith('.pkl')]

# 获取所有Theme名称（从PKL文件中提取）
def get_all_themes(pkl_files):
    themes = set()
    for file_path in pkl_files:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        themes.add(data['metadata']['theme_name'])
    return sorted(list(themes))

# 加载指定模态的PKL文件并获取所有Theme
pkl_files = get_pkl_files(PKL_2D_DIR) if MODALITY == '2d' else get_pkl_files(PKL_3D_DIR)
all_themes = get_all_themes(pkl_files)

print(f"当前选择模态：{MODALITY}")
print(f"找到的PKL文件数：{len(pkl_files)}")
print(f"所有可用Theme：{all_themes}")

# 验证文件存在
assert len(pkl_files) > 0, f"未找到{MODALITY}模态的PKL文件，请检查路径"
assert len(all_themes) > 0, "未从PKL文件中提取到Theme信息"

当前选择模态：3d
找到的PKL文件数：11
所有可用Theme：['arm_swing_as', 'body_swing_bs', 'chest_expansion_ce', 'drumming_dr', 'frog_pose_fg', 'maracas_forward_shaking_mfs', 'maracas_shaking_ms', 'sing_and_clap_sac', 'squat_sq', 'tree_pose_tr', 'twist_pose_tw']


In [21]:
class AnomalySkeletonDataset(Dataset):
    """按Theme分组的骨架数据集（帧级别拆分，保留每个帧的sample归属信息）"""
    def __init__(self, pkl_files, modality='3d', target_theme=None, max_frames=32):
        self.modality = modality
        self.skeleton_key = 'skeleton_2d' if modality == '2d' else 'skeleton_3d'
        self.max_frames = max_frames
        self.target_theme = target_theme  # 目标Theme（单独为该Theme训练正常模型）
        self.frame_data = []  # 帧级别数据：每个元素是单帧骨架
        self.frame_info = []  # 帧的元信息（核心：保留sample级归属）
        
        # 加载目标Theme的所有样本（仅用该Theme的样本训练“正常模型”）
        sample_idx = 0  # 全局sample索引（标记每个sample的唯一ID）
        for file_path in pkl_files:
            with open(file_path, 'rb') as f:
                data = pickle.load(f)
            
            theme_name = data['metadata']['theme_name']
            if theme_name != target_theme:
                continue  # 只加载目标Theme的数据
            
            print(f"加载目标Theme：{theme_name}，样本数：{data['metadata']['sample_count']}")
            for sample in data['samples']:
                skeleton = sample[self.skeleton_key]  # (T, J, C)：帧数×关节数×通道数
                motion_name = sample['motion_name']
                T = skeleton.shape[0]
                
                # 帧级别拆分：将每个sample的每帧单独作为一个数据点，保留完整归属信息
                for frame_idx in range(T):
                    frame = skeleton[frame_idx]  # (J, C)：单帧骨架
                    
                    # 记录元信息（关键：标注sample_idx、motion_name、frame_idx，便于定位）
                    self.frame_info.append({
                        'sample_idx': sample_idx,  # 每个sample的唯一ID
                        'motion_name': motion_name,  # sample的名称
                        'frame_idx': frame_idx,  # 该帧在sample中的索引（0开始）
                        'theme': theme_name,
                        'is_anomaly': False,  # 初始标记为正常
                        'reconstruction_error': 0.0,  # 后续填充重建误差
                        'anomaly_score': 0.0,  # 后续填充异常分数（0-1）
                        'anomaly_prob': 0.0  # 后续填充异常概率（0-1，越接近1越可能异常）
                    })
                    self.frame_data.append(frame)
                
                sample_idx += 1  # 每个sample对应一个唯一索引
        
        # 数据归一化（按关节和通道归一化，提升模型稳定性）
        self.frame_data = np.array(self.frame_data, dtype=np.float32)  # (N, J, C)
        self.mean = np.mean(self.frame_data, axis=0, keepdims=True)
        self.std = np.std(self.frame_data, axis=0, keepdims=True) + 1e-6  # 避免除零
        self.frame_data = (self.frame_data - self.mean) / self.std
        
        # 统计sample信息
        self.sample_count = sample_idx  # 总sample数
        self.frame_info_df = pd.DataFrame(self.frame_info)
        
        print(f"\n目标Theme：{target_theme}")
        print(f"总Sample数：{self.sample_count}")
        print(f"总帧数：{len(self.frame_data)}")
        print(f"帧数据形状：{self.frame_data.shape}（帧数×关节数×通道数）")
        print(f"每个Sample的平均帧数：{len(self.frame_data)/self.sample_count:.1f}")

    def __len__(self):
        return len(self.frame_data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.frame_data[idx], dtype=torch.float32)

# 为每个Theme创建单独的数据集（后续逐一训练）
theme_datasets = {}
for theme in all_themes:
    dataset = AnomalySkeletonDataset(
        pkl_files=pkl_files,
        modality=MODALITY,
        target_theme=theme,
        max_frames=MAX_FRAMES
    )
    if len(dataset) > 0:  # 只保留有数据的Theme
        theme_datasets[theme] = dataset

print(f"\n有效Theme（有数据）：{list(theme_datasets.keys())}")
assert len(theme_datasets) > 0, "无有效Theme数据，无法继续训练"

加载目标Theme：arm_swing_as，样本数：105

目标Theme：arm_swing_as
总Sample数：105
总帧数：3360
帧数据形状：(3360, 71, 3)（帧数×关节数×通道数）
每个Sample的平均帧数：32.0
加载目标Theme：body_swing_bs，样本数：119

目标Theme：body_swing_bs
总Sample数：119
总帧数：3808
帧数据形状：(3808, 71, 3)（帧数×关节数×通道数）
每个Sample的平均帧数：32.0
加载目标Theme：chest_expansion_ce，样本数：114

目标Theme：chest_expansion_ce
总Sample数：114
总帧数：7296
帧数据形状：(7296, 71, 3)（帧数×关节数×通道数）
每个Sample的平均帧数：64.0
加载目标Theme：drumming_dr，样本数：545

目标Theme：drumming_dr
总Sample数：545
总帧数：17440
帧数据形状：(17440, 71, 3)（帧数×关节数×通道数）
每个Sample的平均帧数：32.0
加载目标Theme：frog_pose_fg，样本数：113

目标Theme：frog_pose_fg
总Sample数：113
总帧数：3616
帧数据形状：(3616, 71, 3)（帧数×关节数×通道数）
每个Sample的平均帧数：32.0
加载目标Theme：maracas_forward_shaking_mfs，样本数：103

目标Theme：maracas_forward_shaking_mfs
总Sample数：103
总帧数：6592
帧数据形状：(6592, 71, 3)（帧数×关节数×通道数）
每个Sample的平均帧数：64.0
加载目标Theme：maracas_shaking_ms，样本数：130

目标Theme：maracas_shaking_ms
总Sample数：130
总帧数：8320
帧数据形状：(8320, 71, 3)（帧数×关节数×通道数）
每个Sample的平均帧数：64.0
加载目标Theme：sing_and_clap_sac，样本数：113

目标Theme：sing_and_clap_sac

In [22]:
class FrameAutoencoder(nn.Module):
    """帧级别骨架自编码器：学习正常帧的特征，通过重建误差检测异常"""
    def __init__(self, input_channels=3, num_joints=71, hidden_dim=128):
        super().__init__()
        self.input_dim = num_joints * input_channels  # 扁平化输入（J×C）
        self.hidden_dim = hidden_dim
        
        # 编码器：压缩特征（帧骨架 → 低维特征）
        self.encoder = nn.Sequential(
            nn.Flatten(),  # (B, J, C) → (B, J×C)
            nn.Linear(self.input_dim, hidden_dim * 2),
            nn.LayerNorm(hidden_dim * 2),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, hidden_dim // 2)  # 瓶颈层
        )
        
        # 解码器：重建帧（低维特征 → 原始帧骨架）
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.LayerNorm(hidden_dim * 2),
            nn.GELU(),
            nn.Linear(hidden_dim * 2, self.input_dim),
            nn.Unflatten(dim=1, unflattened_size=(num_joints, input_channels))  # (B, J×C) → (B, J, C)
        )
    
    def forward(self, x):
        # 编码 → 解码 → 输出重建结果
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon
    
    def get_reconstruction_error(self, x):
        """计算重建误差（MSE）：异常帧的误差会显著高于正常帧"""
        with torch.no_grad():
            x_recon = self.forward(x)
            mse = torch.mean((x - x_recon) ** 2, dim=[1, 2])  # 按关节和通道求平均
        return mse.cpu().numpy()

# 初始化模型（根据模态选择输入通道数）
input_channels = 2 if MODALITY == '2d' else 3
base_model = FrameAutoencoder(
    input_channels=input_channels,
    num_joints=NUM_JOINTS,
    hidden_dim=128
).to(device)

print(f"自编码器模型结构：")
print(f"输入通道数：{input_channels}（{MODALITY}模态）")
print(f"关节数：{NUM_JOINTS}")
print(f"模型参数总数：{sum(p.numel() for p in base_model.parameters()) / 1e6:.2f}M")

自编码器模型结构：
输入通道数：3（3d模态）
关节数：71
模型参数总数：0.19M


In [30]:
# ========== 新增：训练/测试集划分配置 ==========
TEST_SPLIT_RATIO = 0.2  # 8:2 划分训练/测试集
RANDOM_STATE = 42  # 固定随机种子，确保划分结果可复现

# ========== 批量训练所有Theme的自编码器（含训练/测试集划分）+ 训练集异常检测 ==========
theme_models = {}  # {theme: (model, train_losses, train_dataset, test_dataset)}
theme_anomaly_results = {}  # {theme: (train_frame_info_df, threshold, train_sample_stats)}

for theme, full_dataset in theme_datasets.items():
    print(f"\n" + "="*100)
    print(f"处理Theme：{theme}（含训练/测试集8:2划分）")
    print("="*100)
    
    # -------------------------- 关键修改：划分训练/测试集 --------------------------
    # 获取所有帧的索引，按Sample分组划分（避免同一Sample同时出现在训练/测试集）
    all_sample_ids = full_dataset.frame_info_df['sample_idx'].unique()
    # 按Sample划分训练/测试集（确保同一Sample的所有帧归为同一集合）
    train_sample_ids, test_sample_ids = train_test_split(
        all_sample_ids,
        test_size=TEST_SPLIT_RATIO,
        random_state=RANDOM_STATE,
        shuffle=True
    )
    
    # 筛选训练集数据（帧+元信息）
    train_mask = full_dataset.frame_info_df['sample_idx'].isin(train_sample_ids)
    train_frame_data = full_dataset.frame_data[train_mask]
    train_frame_info_df = full_dataset.frame_info_df[train_mask].copy().reset_index(drop=True)
    
    # 筛选测试集数据（帧+元信息，仅保存，不参与训练）
    test_mask = full_dataset.frame_info_df['sample_idx'].isin(test_sample_ids)
    test_frame_data = full_dataset.frame_data[test_mask]
    test_frame_info_df = full_dataset.frame_info_df[test_mask].copy().reset_index(drop=True)
    
    # 创建训练集Dataset（复用原有结构，仅包含训练集数据）
    class TrainSubsetDataset(Dataset):
        def __init__(self, frame_data, frame_info_df, mean, std):
            self.frame_data = frame_data
            self.frame_info_df = frame_info_df
            self.mean = mean
            self.std = std
            self.sample_count = len(train_sample_ids)
        
        def __len__(self):
            return len(self.frame_data)
        
        def __getitem__(self, idx):
            return torch.tensor(self.frame_data[idx], dtype=torch.float32)
    
    train_dataset = TrainSubsetDataset(
        frame_data=train_frame_data,
        frame_info_df=train_frame_info_df,
        mean=full_dataset.mean,
        std=full_dataset.std
    )
    
    # 创建测试集Dataset（仅保存，供后续测试使用）
    class TestSubsetDataset(Dataset):
        def __init__(self, frame_data, frame_info_df, mean, std):
            self.frame_data = frame_data
            self.frame_info_df = frame_info_df
            self.mean = mean
            self.std = std
            self.sample_count = len(test_sample_ids)
        
        def __len__(self):
            return len(self.frame_data)
        
        def __getitem__(self, idx):
            return torch.tensor(self.frame_data[idx], dtype=torch.float32)
    
    test_dataset = TestSubsetDataset(
        frame_data=test_frame_data,
        frame_info_df=test_frame_info_df,
        mean=full_dataset.mean,
        std=full_dataset.std
    )
    
    # 打印划分结果
    print(f"训练集：Sample数={len(train_sample_ids)} | 帧数={len(train_dataset)}")
    print(f"测试集：Sample数={len(test_sample_ids)} | 帧数={len(test_dataset)}")
    print(f"划分比例：训练集{1-TEST_SPLIT_RATIO:.0%} | 测试集{TEST_SPLIT_RATIO:.0%}")
    
    # -------------------------- 原有训练逻辑（仅用训练集训练） --------------------------
    # 创建训练集DataLoader
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=min(4, os.cpu_count()),
        pin_memory=True if device.type == 'cuda' else False
    )
    
    # 初始化新模型（避免Theme间权重干扰）
    model = FrameAutoencoder(
        input_channels=input_channels,
        num_joints=NUM_JOINTS,
        hidden_dim=128
    ).to(device)
    
    # 训练（仅用训练集）
    trained_model, train_losses = train_autoencoder(
        model=model,
        train_loader=train_loader,
        epochs=EPOCHS,
        lr=1e-4
    )
    
    # -------------------------- 保存模型（新增测试集相关信息） --------------------------
    theme_models[theme] = (trained_model, train_losses, train_dataset, test_dataset)
    model_save_path = os.path.join(OUTPUT_DIR, f"autoencoder_{theme}_{MODALITY}_with_split.pth")
    torch.save({
        'theme': theme,
        'modality': MODALITY,
        'model_state_dict': trained_model.state_dict(),
        'train_losses': train_losses,
        'mean': full_dataset.mean,
        'std': full_dataset.std,
        'train_sample_ids': train_sample_ids.tolist(),
        'test_sample_ids': test_sample_ids.tolist(),
        'test_split_ratio': TEST_SPLIT_RATIO
    }, model_save_path)
    print(f"模型保存路径：{model_save_path}（含训练/测试集划分信息）")
    
    # -------------------------- 仅对训练集做异常检测（测试集留到单元2） --------------------------
    print(f"\n" + "="*80)
    print(f"逐帧检测Theme：{theme}（仅训练集）的异常帧")
    print("="*80)
    train_frame_info_df, threshold, train_sample_stats = detect_anomaly_frames_frame_level(
        theme=theme,
        model=trained_model,
        dataset=train_dataset,  # 仅用训练集检测
        threshold=ANOMALY_THRESHOLD
    )
    theme_anomaly_results[theme] = (train_frame_info_df, threshold, train_sample_stats)
    
    # 保存训练集结果
    train_frame_info_df.to_csv(os.path.join(OUTPUT_DIR, f"{theme}_train_frame_level_results.csv"), index=False, encoding='utf-8-sig')
    train_sample_stats.to_csv(os.path.join(OUTPUT_DIR, f"{theme}_train_sample_anomaly_stats.csv"), encoding='utf-8-sig')
    train_anomaly_frames_df = train_frame_info_df[train_frame_info_df['is_anomaly']].copy()
    train_anomaly_frames_df.to_csv(os.path.join(OUTPUT_DIR, f"{theme}_train_anomaly_frames_detail.csv"), index=False, encoding='utf-8-sig')
    
    print(f"\n训练集结果保存路径：")
    print(f"  - 逐帧详细结果：{theme}_train_frame_level_results.csv")
    print(f"  - Sample异常统计：{theme}_train_sample_anomaly_stats.csv")
    print(f"  - 异常帧详情：{theme}_train_anomaly_frames_detail.csv")
    
    # 保存测试集元信息（供单元2测试使用）
    test_meta_save_path = os.path.join(OUTPUT_DIR, f"{theme}_test_dataset_meta.csv")
    test_dataset.frame_info_df.to_csv(test_meta_save_path, index=False, encoding='utf-8-sig')
    print(f"  - 测试集元信息：{theme}_test_dataset_meta.csv（供后续测试使用）")

# -------------------------- 原有可视化训练损失逻辑 --------------------------
plt.figure(figsize=(12, 6))
for theme, (_, losses, _, _) in theme_models.items():
    plt.plot(range(1, EPOCHS+1), losses, label=theme, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('重建损失（MSE）')
plt.title(f'{MODALITY}模态 - 各Theme自编码器训练损失曲线（8:2划分训练/测试集）')
plt.legend()
plt.grid(alpha=0.3)
plt.savefig(os.path.join(OUTPUT_DIR, 'train_losses_all_themes_with_split.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"\n" + "="*100)
print(f"所有Theme训练完成！")
print(f"关键输出：")
print(f"  1. 模型权重（含划分信息）：autoencoder_{theme}_{MODALITY}_with_split.pth")
print(f"  2. 训练集异常检测结果：{theme}_train_*.csv")
print(f"  3. 测试集元信息：{theme}_test_dataset_meta.csv")
print(f"  4. 训练损失曲线：train_losses_all_themes_with_split.png")
print("="*100)


处理Theme：arm_swing_as（含训练/测试集8:2划分）
训练集：Sample数=84 | 帧数=2688
测试集：Sample数=21 | 帧数=672
划分比例：训练集80% | 测试集20%

开始训练自编码器（共30个Epoch）


Epoch [5/30] | 平均重建损失：0.059854 | LR：0.000093


KeyboardInterrupt: 